In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
from tqdm import tqdm
from scipy.stats import truncnorm, norm, levene, ttest_ind

from numpy.lib.stride_tricks import sliding_window_view

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Conv2D, Embedding, DepthwiseConv2D, SeparableConv2D, Conv2DTranspose, ZeroPadding2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.initializers import GlorotNormal, GlorotUniform, Zeros
from tensorflow.keras import backend as K

from tensorflow.image import extract_patches
# import tensorflow_text as tft

# Conv1D with embedding layer

In [2]:
n_samples = 10000
max_len = 50
vocab_size = 1234
embedding_dim = 500

kernel_size = 3
n_filters = 64

In [3]:
rng = np.random.default_rng(42)
X = rng.integers(0,vocab_size, size=(n_samples, max_len))


model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = max_len, ))
X_emb = model(X)

st=time.time()
model.add(Conv1D(n_filters, kernel_size, input_shape= (max_len, embedding_dim)))
hidden_state = model(X)
et=time.time()
print('running time:',et-st)
cnn = model.layers[1]

running time: 1.0559544563293457


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 500)           617000    
                                                                 
 conv1d (Conv1D)             (None, 48, 64)            96064     
                                                                 
Total params: 713,064
Trainable params: 713,064
Non-trainable params: 0
_________________________________________________________________


In [5]:
output_axis1 = int((max_len-kernel_size)/cnn.strides[0] +1)
print(output_axis1 == cnn.output_shape[1])

True


## implementation: with python for loop

In [298]:
X_emb_ = copy.deepcopy(X_emb.numpy())

st=time.time()
result_all = np.zeros((n_samples, output_axis1, n_filters))

for d in tqdm(range(n_filters)):
    kernel_sample = copy.deepcopy(cnn.kernel[:,:,d].numpy())
    result = np.array([]).reshape(-1, output_axis1)
    for samp in range(n_samples):
        array = [] #np.zeros(output_axis1)
    #     X_emb__ = X_emb_[samp,:,:]
        for k in range(0, output_axis1, cnn.strides[0]):
            array.append((X_emb_[samp,k:k+kernel_size,:]*kernel_sample).sum())
        result = np.vstack((result, array ))
        
    result_all[:,:,d]= result
et=time.time()
print(et-st)

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [02:18<00:00,  2.17s/it]

138.61507439613342


In [303]:
print('Validation:', np.allclose(result_all, hidden_state.numpy(), atol=1e-7))

Validation: True


## implementation: with numpy

In [35]:
X_emb.shape

TensorShape([10000, 50, 500])

In [42]:
st=time.time()
X_emb_sld = sliding_window_view(X_emb, window_shape=cnn.kernel.shape[:2], axis=(1,2))
X_emb_sld = tf.cast(X_emb_sld, tf.float32)
result = tf.einsum('abijk,jkc->abc', X_emb_sld, cnn.kernel)

et=time.time()
print(et-st)

20.20397400856018


In [43]:
X_emb_sld.shape

TensorShape([10000, 48, 1, 3, 500])

In [44]:
st=time.time()
X_emb_sld = sliding_window_view(X_emb, window_shape=cnn.kernel.shape[:2], axis=(1,2))
X_emb_sld_flt = X_emb_sld.reshape(n_samples,output_axis1,1,-1,)
kernel_flt = tf.reshape(cnn.kernel, [-1, n_filters] )
result = tf.einsum('aijk,kl->ail', X_emb_sld_flt, kernel_flt)

et=time.time()
print(et-st)

3.0047667026519775


In [45]:
print('Validation:', np.alltrue(result == hidden_state))

Validation: True


## implementation: with tensorflow

In [46]:
st=time.time()
X_emb_sld = tft.sliding_window(X_emb, kernel_size, axis=1)

result = tf.einsum('abjk,jkc->abc', X_emb_sld, cnn.kernel)

et=time.time()
print(et-st)

1.912224292755127


In [47]:
print('Validation:', np.alltrue(result == hidden_state))

Validation: True


In [48]:
st=time.time()
X_emb_sld = tft.sliding_window(X_emb, kernel_size, axis=1)
X_emb_sld_flt = tf.reshape(X_emb_sld, [X_emb_sld.shape[0], X_emb_sld.shape[1], -1,])
kernel_flt = tf.reshape(cnn.kernel, [-1, n_filters] )
result = tf.einsum('aik,kl->ail', X_emb_sld_flt, kernel_flt)

et=time.time()
print(et-st)

1.9849357604980469


In [49]:
print('Validation:', np.alltrue(result == hidden_state ))

Validation: True


# Conv2D

## Create sample data

In [370]:
num_samples = 16
image_size = (256, 256)
num_channels = 5

kernel_size = (5, 5)
strides = 2
n_filters = 64

In [371]:
rng = np.random.default_rng(42)
X = rng.integers(0, 256, size=(num_samples,)+image_size+(num_channels,))

X = tf.cast(X, tf.float32)

In [372]:
X.shape

TensorShape([16, 256, 256, 5])

## Conventional Convolution

In [764]:
strides = 3

In [765]:
st = time.time()

model = Sequential()
model.add(Conv2D(n_filters, kernel_size, input_shape = image_size+(num_channels,), strides = strides, padding='same', ) )
X_ = model(X)

et = time.time()
print('running time:', et-st)

running time: 0.04082131385803223


In [766]:
model.summary()

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 86, 86, 64)        8064      
                                                                 
Total params: 8,064
Trainable params: 8,064
Non-trainable params: 0
_________________________________________________________________


In [768]:
cnn = model.layers[0]

In [769]:
cnn.kernel.shape

TensorShape([5, 5, 5, 64])

In [770]:
print('# of params:', cnn.count_params())
cnn.count_params() == np.product(cnn.kernel.shape, axis=0) + cnn.bias.shape[0]

# of params: 8064


True

In [771]:
pad = 2 if cnn.padding == 'same' else 0

print('shape of feature map:',cnn.output_shape[1:3])
cnn.output_shape[1:3] == (int((image_size[0] - kernel_size[0] + pad)/strides + 1), int((image_size[1] - kernel_size[1] + pad)/strides + 1))

shape of feature map: (86, 86)


False

### implement 1

In [772]:
st = time.time()
# cnn = model.layers[0]
ep = extract_patches(X, sizes= [1]+cnn.kernel.shape[:2]+[1], strides=[1, strides, strides, 1], rates = [1,1,1,1], padding = cnn.padding.upper() )
kf = tf.reshape(cnn.kernel, [-1, n_filters])

result = tf.einsum('abci,id->abcd', ep, kf)

et = time.time()
print('running time:', et-st)

running time: 0.06569194793701172


In [773]:
ep.shape, kf.shape, result.shape

(TensorShape([16, 86, 86, 125]),
 TensorShape([125, 64]),
 TensorShape([16, 86, 86, 64]))

In [780]:
np.alltrue(result == X_)

True

### implement 2

In [782]:
st = time.time()
# cnn = model.layers[0]
ep = extract_patches(X, sizes= [1]+cnn.kernel.shape[:2]+[1], strides=[1, strides, strides, 1], rates = [1,1,1,1], padding = cnn.padding.upper() )
ep_shape = ep.shape
ep_ = tf.reshape(ep, list(ep_shape[:3]) + list(kernel_size) + [-1])

result = tf.einsum('abcijk,ijkd->abcd', ep_, cnn.kernel)

et = time.time()
print('running time:', et-st)

running time: 0.06233692169189453


In [783]:
ep.shape

TensorShape([16, 86, 86, 125])

In [784]:
np.alltrue(result == X_)

True

## Depthwise Convolution

* `input_shape`: $\text{batch size}\times \text{input width}\times \text{input height} \times \text{num of input channels}$
* `kernel_shape`: $\text{kernel width}\times \text{kernel height} \times \text{num of input channels}$

$\large \displaystyle I_j * K_j = O_j$ w.r.t the j-th input channel

$\large \displaystyle O = \begin{bmatrix} O_1\\ \vdots\\ O_n \end{bmatrix}$

In [51]:
st = time.time()

model = Sequential()
model.add(DepthwiseConv2D(kernel_size, strides = strides, padding='valid', input_shape = image_size+(num_channels,), depthwise_initializer='ones' ) )
X_ = model(X)

et = time.time()
print('running time:', et-st)

running time: 0.060999393463134766


In [52]:
X_.shape

TensorShape([64, 127, 127, 7])

In [53]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 depthwise_conv2d_2 (Depthwi  (None, 127, 127, 7)      70        
 seConv2D)                                                       
                                                                 
Total params: 70
Trainable params: 70
Non-trainable params: 0
_________________________________________________________________


In [54]:
dcnn = model.layers[0]

In [55]:
dcnn.depthwise_kernel.shape

TensorShape([3, 3, 7, 1])

In [56]:
st = time.time()

ep = extract_patches(X, sizes= [1]+dcnn.depthwise_kernel.shape[:2]+[1], strides=[1, strides, strides, 1], rates = [1,1,1,1], padding = 'VALID' )
ep_ = tf.reshape(ep, list(X_.shape[:3]) + list(kernel_size) + [-1])

kf = tf.squeeze(dcnn.depthwise_kernel, axis=-1)

result = tf.einsum('abcdef,def->abcf', ep_, kf)

et = time.time()
print('running time:', et-st)

running time: 0.4218616485595703


In [57]:
# ep_ shape: batch size * f.map width * f.map height * kernel width * kernel height * channels

ep_.shape, kf.shape, result.shape

(TensorShape([64, 127, 127, 3, 3, 7]),
 TensorShape([3, 3, 7]),
 TensorShape([64, 127, 127, 7]))

In [58]:
np.alltrue(result == X_)

True

## Depthwise separable convolution

* Depthwise separable: **Depthwise** convolution -> **Pointwise** convolution 

In [59]:
tf.random.set_seed(1)

model = Sequential()
model.add(SeparableConv2D(64, kernel_size, strides = strides, padding='valid', input_shape = image_size+(num_channels,) ) )
X_dsc = model(X)

In [60]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d (Separable  (None, 127, 127, 64)     575       
 Conv2D)                                                         
                                                                 
Total params: 575
Trainable params: 575
Non-trainable params: 0
_________________________________________________________________


In [61]:
X_.shape

TensorShape([64, 127, 127, 7])

In [62]:
scnn = model.layers[0]

In [63]:
tf.random.set_seed(1)

model = Sequential()
model.add(DepthwiseConv2D(kernel_size, strides = strides, padding='valid', input_shape = image_size+(num_channels,), use_bias=False ) )
model.add(Conv2D(64, 1, strides = 1, padding='valid',  ) )
X_dsc_ = model(X)

In [64]:
dwcnn, pwcnn = model.layers

In [65]:
np.alltrue(dwcnn.depthwise_kernel == scnn.depthwise_kernel)

True

In [66]:
np.alltrue(pwcnn.kernel == scnn.pointwise_kernel)

True

In [67]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 depthwise_conv2d_3 (Depthwi  (None, 127, 127, 7)      63        
 seConv2D)                                                       
                                                                 
 conv2d_2 (Conv2D)           (None, 127, 127, 64)      512       
                                                                 
Total params: 575
Trainable params: 575
Non-trainable params: 0
_________________________________________________________________


In [68]:
np.alltrue(X_dsc == X_dsc_)

True